In [15]:
"""Solve a nonlinear Poisson's equation,

    div (1 + u^2) grad u(x, y) = f(x, y)

and boundary conditions given by

    u(x, y) = 1

This is equivalent to solving the variational problem

    F(u) = ((1 + u^2)*grad(u), grad(v)) + (f, v) = 0

Ref:
- https://fenicsproject.org/olddocs/dolfin/1.3.0/python/demo/documented/poisson/python/documentation.html
- https://fenicsproject.org/olddocs/dolfin/1.3.0/python/demo/documented/nonlinear-poisson/python/documentation.html
- https://github.com/nw2190/ConvPDE/blob/master/Nonlinear_Poisson/Setup/solver.py
"""

import matplotlib.pyplot as plt
import meshio
import numpy as np
from dolfin import *
from scipy import interpolate, ndimage

from spaces import GRF2D


class MyExpression(UserExpression):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        x = np.linspace(0, 1, num=101)
        y = np.linspace(0, 1, num=101)
        f0 = x[:, None] * np.sin(y)
        # f0 = np.exp(-((x[:, None] - 0.5) ** 2 + (y - 0.5) ** 2) / 0.02)

        # f0
        # f = f0
        # Random f
        # np.random.seed(0)
        f = 2 * np.random.rand(101, 101) - 1
        f += f0
        # f = ndimage.gaussian_filter(f, 1)
        # New f
#         f = f0 + 0.05
        # New random f
#         space = GRF2D(length_scale=0.1, N=101)
#         feature = space.random(1)
#         xv, yv = np.meshgrid(x, y)
#         sensors = np.vstack((np.ravel(xv), np.ravel(yv))).T
#         f = space.eval_u(feature, sensors)[0]
#         f = np.reshape(f, (len(y), len(x))).T
#         f = f0 + 0.05 * f

        self.interp = interpolate.RegularGridInterpolator((x, y), f)

    def _eval(self, x):
        # return x[0] * np.sin(x[1])  # f0
        return self.interp(x)[0]

    def eval(self, value, x):
        value[0] = self._eval(x)

    def eval_batch(self, x):
        return np.array(list(map(self._eval, x)))


def solver(f):
    # Create mesh and define function space
    mesh = UnitSquareMesh(100, 100)
    # mesh = UnitSquareMesh.create(100, 100, CellType.Type.Quadrilateral)
    # plot(mesh, interactive=True)
    # File("data/mesh.pvd") << mesh

    V = FunctionSpace(mesh, "CG", 1)

    # Define boundary condition
    g = Constant(0.0)
    bc = DirichletBC(V, g, lambda _, on_boundary: on_boundary)

    # Linear solver
    # u = TrialFunction(V)
    # v = TestFunction(V)
    # a = inner(grad(u), grad(v)) * dx
    # L = -10 * f * v * dx
    # u = Function(V)
    # solve(a == L, u, bc)

    # Nonlinear solver
    u = Function(V)
    v = TestFunction(V)
    # F = inner(grad(u), grad(v)) * dx + 10 * f * v * dx
    F = inner((1 + u ** 2) * grad(u), grad(v)) * dx + 10 * f * v * dx
    solve(F == 0, u, bc)

    # Save solution in VTK format
    file = File("data/u.pvd")
    file << u


def convert(f):
    mesh = meshio.read("data/u000000.vtu")
    x = mesh.points[:, :2]
#     u = mesh.point_data["f_8"][:, None]
    key_pt = list(mesh.point_data.keys())[0]
    u = mesh.point_data[key_pt][:, None]
    # np.savetxt("data/u.dat", np.hstack((x, u)))
    np.savetxt("data/u_random.dat", u.reshape(101, 101))  # Ny x Nx
    f = f.eval_batch(x)[:, None]
    # np.savetxt("data/f.dat", np.hstack((x, f)))
    np.savetxt("data/f_random.dat", f.reshape(101, 101))


def main():
    # f = Expression("x[0]*sin(x[1])", degree=2)
    f = MyExpression(degree=2)
    solver(f)
    convert(f)


if __name__ == "__main__":
    main()
